In [1]:
import sys
sys.path.append("../src")
from utils import *

In [2]:
import muon as mu

In [3]:
adata = mu.read("/data/projects/dschaub/anca-project/data/anca/exploratory/ANCA_27PK27PB_T_harmony_r_annotated.h5mu")

/opt/conda/lib/python3.9/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


In [4]:
adata = adata["rna"]

In [5]:
adata.X.max()

8.72714754694824

In [6]:
cell_types = ["CD4+ EM/RM", "CD8+ EM/RM"]
adata.shape

(72416, 21419)

In [7]:
adata.obs["cell_type"] = adata.obs["cell_type_v1"].tolist()
adata.obs["cell_type"].replace({"CD4+ Teff": "EM/RM",
                                   "CD8+ Teff": "EM/RM"},
                                  inplace=True)

In [8]:
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

In [9]:
import pickle
pickle_path = os.path.join(datadir, "filtered_added.pkl")
with open(pickle_path, "rb") as handle:
    newdict = pickle.load(handle)

In [10]:
import drug2cell as d2c
d2c.score(adata, targets=newdict['L'], nested=False, use_raw=False)

In [11]:
sc.tl.rank_genes_groups(adata.uns['drug2cell'], method="wilcoxon", groupby="cell_type", pts=True, use_raw=False, 
                        key_added="wilcoxon_drug")
# sc.tl.rank_genes_groups(adata.uns['drug2cell'], method="wilcoxon", groupby="cell_type_yu", pts=True, use_raw=False, 
#                         key_added="wilcoxon_drug_old")

In [12]:
log2fc = 0.25
pval = 0.05

In [13]:
dfs = {}
dfs_old = {}
for clst in adata.obs["cell_type"].unique(): #_merge
    dfs[clst] = sc.get.rank_genes_groups_df(adata.uns["drug2cell"], group=clst, pval_cutoff=pval,
                                            #log2fc_min=log2fc,
                                            key="wilcoxon_drug").set_index("names")
# for clst in adata.obs["cell_type_yu"].unique(): #_merge
#     dfs_old[clst] = sc.get.rank_genes_groups_df(adata.uns["drug2cell"], group=clst, pval_cutoff=pval,
#                                             #log2fc_min=log2fc,
#                                             key="wilcoxon_drug_old").set_index("names")
    # spec = pd.read_csv(os.path.join(datadir, "cd4cd8_EMRM_scores.csv"), index_col=0)

In [14]:
dfs.keys()

dict_keys(['stressed T cells', 'EM/RM', 'CD4+ naive', 'MAIT', 'Treg', 'NKT/CTL', 'CD8+ Tem', 'CD4+ Tcm', 'NKT', 'gdT', 'NK', 'Prolif.', 'CD4+/CD8+ Tcm', 'CD8+ naive'])

In [15]:
sns.set(style="ticks", font_scale=1)

In [17]:
pct_group = 0.75
pct_ref = 0.5

for clst in ["EM/RM"]: 
    dfs[clst] = dfs[clst][dfs[clst]["pct_nz_group"]>=pct_group]
    # dfs_old[clst] = dfs_old[clst][dfs_old[clst]["pct_nz_group"]>=pct_group]

In [19]:
dfs[clst] = dfs[clst].sort_values(by="logfoldchanges", ascending=False)
# dfs_old[clst] = dfs_old[clst].sort_values(by="logfoldchanges", ascending=False)

In [ ]:
dfs["EM/RM"].to_csv(os.path.join(datadir, "drug_scores_single_cell_EMRM_1.csv"))

In [20]:
dfs["EM/RM"] - pd.read_csv(os.path.join(datadir, "drug_scores_single_cell_EMRM.csv"), index_col=0)

,scores,logfoldchanges,pvals,pvals_adj,pct_nz_group,pct_nz_reference
names,,,,,,
ADALIMUMAB,-3.178711e-07,4.259605e-09,0.000000e+00,-4.469445e-249,0.000000e+00,-1.110223e-16
ALDESLEUKIN,-4.565430e-08,-1.662254e-09,2.330731e-156,-1.864585e-155,0.000000e+00,0.000000e+00
ALEFACEPT,2.387695e-07,-4.924622e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BASILIXIMAB,-4.565430e-08,-1.662254e-09,2.330731e-156,-1.864585e-155,0.000000e+00,0.000000e+00
BELANTAMAB MAFODOTIN,3.896484e-07,8.113098e-10,0.000000e+00,0.000000e+00,1.110223e-16,0.000000e+00
BELUMOSUDIL,-2.717285e-07,3.387527e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BEVACIZUMAB,-3.166504e-06,-1.065063e-09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
BRENTUXIMAB VEDOTIN,1.982422e-06,-8.668518e-10,0.000000e+00,0.000000e+00,1.110223e-16,0.000000e+00
CAPECITABINE,4.893799e-07,1.105194e-09,0.000000e+00,-6.379005e-204,0.000000e+00,0.000000e+00


,scores,logfoldchanges,pvals,pvals_adj,pct_nz_group,pct_nz_reference
names,,,,,,
ALEFACEPT,63.183357,1.068921,0.000000e+00,0.000000e+00,0.828350,0.659190
CELECOXIB,86.967620,0.969703,0.000000e+00,0.000000e+00,0.894936,0.729000
PLERIXAFOR,90.013600,0.854371,0.000000e+00,0.000000e+00,0.950189,0.830414
ENFORTUMAB VEDOTIN,68.294050,0.692203,0.000000e+00,0.000000e+00,0.936872,0.859324
VINFLUNINE,68.294050,0.692203,0.000000e+00,0.000000e+00,0.936872,0.859324
BRENTUXIMAB VEDOTIN,68.320280,0.685389,0.000000e+00,0.000000e+00,0.936872,0.859376
BELANTAMAB MAFODOTIN,68.280624,0.685355,0.000000e+00,0.000000e+00,0.936872,0.859324
TRASTUZUMAB EMTANSINE,67.849174,0.679666,0.000000e+00,0.000000e+00,0.937128,0.860732
POLATUZUMAB VEDOTIN,67.322130,0.668754,0.000000e+00,0.000000e+00,0.939561,0.866173
